# Local Binary Pattern for Image Texture Classfication

In [358]:
# Imports 
import os
import glob2 as glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report


## Configuration

In [309]:
dataset_folder = 'dataset\\Splited' # INSERT THE PATH TO THE DATASET

# Define the paths to the train and test folders
train_folder = os.path.join(dataset_folder, 'train')
test_folder = os.path.join(dataset_folder, 'valid')

# Retrieve the list of file names in the train folder
train_classes_path = glob.glob(os.path.join(train_folder, "*"))

# Retrieve the list of file names in the test folder
validation_classes_path = glob.glob(os.path.join(test_folder, "*"))

## Distance Metrics

In [399]:
# Chi-square distance
def chi_square_distance(hist_P, hist_Q):
    # Ensure histograms are not zero
    hist_P[hist_P == 0] = 1e-10
    hist_Q[hist_Q == 0] = 1e-10
    
    # Calculate the Chi-square distance
    distance = np.sum((hist_P - hist_Q)**2 / (hist_P + hist_Q))
    
    return distance

def chi2_distance(A, B):
 
    # compute the chi-squared distance using above formula
    chi = 0.5 * np.sum([((a - b) ** 2) / (a + b) 
                      for (a, b) in zip(A, B)])

    return chi

## Calculate LBP

In [258]:
def calculate_lbp(image): # custom implementation based on the adjacent negighbours
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Initialize the LBP image
    lbp_image = np.zeros_like(gray)
    pad_width = 1
    gray = np.pad(gray, pad_width, mode='constant', constant_values=0)
    
    # Define 8 neighbors for each pixel
    neighbors = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    
    # Iterate over each pixel in the image
    for i in range(1, gray.shape[0] - 1):
        for j in range(1, gray.shape[1] - 1):
            # Get the binary pattern
            binary_pattern = ''
            center_value = gray[i, j]
            for dy, dx in neighbors:
                neighbor_value = gray[i + dy, j + dx]
                binary_pattern += '1' if neighbor_value >= center_value else '0'
            
            # Convert binary pattern to decimal
            lbp_value = int(binary_pattern, 2)
            # print(lbp_value)
            
            # Assign LBP value to corresponding pixel in LBP image
            lbp_image[i-1, j-1] = lbp_value
    
    return lbp_image

## Funcions to compare an image with an available class

In [416]:
def find_minimum_distance(image_hist, train_class_path, representations):
    distances = []
    train_files_path = glob.glob(os.path.join(train_class_path, "*"))
    for train_image_path in train_files_path:

        train_image_hist = representations[train_image_path]

        distance = chi_square_distance(image_hist, train_image_hist)
        distances.append(distance)

    distances = np.array(distances)
    index_of_min = np.argmin(distances)
    file_min = train_files_path[index_of_min]
    return distances.min(), file_min

In [415]:
def classify_image(image_path, train_classes_path, representations, representations_val):

    image_hist = representations_val[image_path]

    minimum_distances = []
    file_mins = []

    for train_class_path in train_classes_path:
        min_distance_from_class, file_min = find_minimum_distance(image_hist, train_class_path, representations)
        minimum_distances.append(min_distance_from_class)
        file_mins.append(file_min)

    minimum_distances = np.array(minimum_distances)
    # print(minimum_distances)
    
    file_mins = np.array(file_mins) # the files that are most similar among the class members in the training set
    # print(file_mins)
    
    class_index = np.argmin(minimum_distances)

    return file_mins[class_index]

## Generate the LPB histogram representation of the training samples

In [419]:
def generate_lpb_representations(train_classes_path1):
    representations = {}

    for train_class_path in train_classes_path1:
        train_files_path = glob.glob(os.path.join(train_class_path, "*"))
        for train_image_path in train_files_path: 
            # read image
            train_image = cv2.imread(train_image_path)
            # compute the LBP for the two images
            train_image_lbp = calculate_lbp(image2).ravel()

            representations[train_image_path] = train_image_lbp

    return representations

In [420]:
representations = generate_lpb_representations(train_classes_path)

In [434]:
representations_val = generate_lpb_representations(validation_classes_path)

## Run the comparison for all validation samples

In [ ]:
predictions_lbp = []
for sample in representations_val:
    prediction = classify_image(sample, train_classes_path, representations, representations_val)
    predictions_lbp.append(prediction)

In [404]:
predictions_lbp = [p.split("\\")[-2] for p in predictions_lbp]

In [405]:
actual_classes = [a.split("\\")[-2] for a in representations_val]

In [406]:
print(classification_report(actual_classes,predictions_lbp,zero_division=np.nan))

                      precision    recall  f1-score   support

  KTH_aluminium_foil       0.05      1.00      0.09       216
     KTH_brown_bread        nan      0.00       nan        41
        KTH_corduroy        nan      0.00       nan       256
            KTH_cork        nan      0.00       nan       216
          KTH_cotton        nan      0.00       nan       257
         KTH_cracker        nan      0.00       nan        34
           KTH_linen        nan      0.00       nan       257
     KTH_orange_peel        nan      0.00       nan        25
          KTH_sponge        nan      0.00       nan        41
       KTH_styrofoam        nan      0.00       nan        41
            KTH_wool        nan      0.00       nan       216
    Kyberge_blanket1        nan      0.00       nan        80
    Kyberge_blanket2        nan      0.00       nan        80
     Kyberge_canvas1        nan      0.00       nan        80
    Kyberge_ceiling1        nan      0.00       nan        80
    Kyb